In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import scipy.spatial.distance as ssd
from scipy.cluster import hierarchy
import matplotlib.pyplot as plt
from scipy import stats
pd.options.mode.chained_assignment = None 

In [2]:
#read file as pandas dataframe
data = pd.read_csv('variant_surveillance.tsv', sep = '\t',dtype = {'Variant': 'string','Is reference?': 'string'})
data = data[data['Clade'].notna()]
data.head()

/tmp/ipykernel_145191/797462131.py:2: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('variant_surveillance.tsv', sep = '\t',dtype = {'Variant': 'string','Is reference?': 'string'})


,Accession ID,Type,Clade,Pango lineage,Pangolin version,AA Substitutions,Variant,Collection date,Location,Host,Submission date,Is reference?,Is complete?,Is high coverage?,Is low coverage?,N-Content,GC-Content,Sequence length
0,EPI_ISL_10010236,betacoronavirus,GRA,BA.1.17,PANGO-v1.8,"(NSP5_P132H,Spike_H69del,Spike_T95I,Spike_A67V...",VOC Omicron GRA (B.1.1.529+BA.*) first detecte...,2022-01-25,Oceania / Australia / Australian Capital Terri...,Human,2022-02-17,<NA>,True,NaN,NaN,NaN,0.379635,29747
1,EPI_ISL_10010227,betacoronavirus,GRA,BA.1.17,PLEARN-v1.8,"(NSP5_P132H,Spike_H69del,Spike_T95I,Spike_A67V...",VOC Omicron GRA (B.1.1.529+BA.*) first detecte...,2022-01-25,Oceania / Australia / Australian Capital Terri...,Human,2022-02-17,<NA>,True,NaN,NaN,0.009446,0.379794,29747
2,EPI_ISL_10010232,betacoronavirus,GRA,BA.1.17,PLEARN-v1.8,"(NSP5_P132H,Spike_H69del,Spike_T95I,Spike_A67V...",VOC Omicron GRA (B.1.1.529+BA.*) first detecte...,2022-01-25,Oceania / Australia / Australian Capital Terri...,Human,2022-02-17,<NA>,True,NaN,NaN,0.000134,0.379585,29747
3,EPI_ISL_3578085,betacoronavirus,GK,B.1.617.2,PLEARN-v1.8,"(N_D377Y,M_I82T,Spike_D950N,N_R203M,NSP12_G671...",VOC Delta GK (B.1.617.2+AY.*) first detected i...,2021-08-18,Europe / Turkey,Human,2021-08-23,<NA>,True,True,NaN,0.000101,0.379853,29754
4,EPI_ISL_10299251,betacoronavirus,GRA,BA.1,PLEARN-v1.8,"(NSP5_P132H,Spike_T95I,Spike_A67V,Spike_K417N,...",VOC Omicron GRA (B.1.1.529+BA.*) first detecte...,2022-01-17,Europe / Germany / North Rhine-Westphalia,Human,2022-02-24,<NA>,True,NaN,NaN,0.048908,0.380371,29729


In [9]:
# #clean dataset and change collection date to datetime object
# data = data[['Clade', 'Location', 'Collection date']]
# data['Collection date'] =  pd.to_datetime(data['Collection date'])#.dt.strftime('%Y/%m')
# time_frame = (data['Collection date'] >= '2020/01/01')
# data = data.loc[time_frame]
# data.head()

,Clade,Location,Collection date
0,GRA,Oceania / Australia / Australian Capital Terri...,2022-01-25
1,GRA,Oceania / Australia / Australian Capital Terri...,2022-01-25
2,GRA,Oceania / Australia / Australian Capital Terri...,2022-01-25
3,GK,Europe / Turkey,2021-08-18
4,GRA,Europe / Germany / North Rhine-Westphalia,2022-01-17


In [10]:
# data['Date']= data['Collection date'].dt.to_period('W')
# data.head()

,Clade,Location,Collection date,Date
0,GRA,Oceania / Australia / Australian Capital Terri...,2022-01-25,2022-01-24/2022-01-30
1,GRA,Oceania / Australia / Australian Capital Terri...,2022-01-25,2022-01-24/2022-01-30
2,GRA,Oceania / Australia / Australian Capital Terri...,2022-01-25,2022-01-24/2022-01-30
3,GK,Europe / Turkey,2021-08-18,2021-08-16/2021-08-22
4,GRA,Europe / Germany / North Rhine-Westphalia,2022-01-17,2022-01-17/2022-01-23


In [43]:
#clean dataset and change collection date to datetime object
data = data[['Clade', 'Location', 'Collection date']]
data['Collection date'] =  pd.to_datetime(data['Collection date']).dt.strftime('%Y/%m')
time_frame = (data['Collection date'] >= '2020/01')
data = data.loc[time_frame]
data.head()

,Clade,Location,Collection date
0,GRA,Oceania / Australia / Australian Capital Terri...,2022/01
1,GRA,Oceania / Australia / Australian Capital Terri...,2022/01
2,GRA,Oceania / Australia / Australian Capital Terri...,2022/01
3,GK,Europe / Turkey,2021/08
4,GRA,Europe / Germany / North Rhine-Westphalia,2022/01


In [44]:
df = data.copy()

In [45]:
#specify strings for each continent
df.loc[df['Location'].str.contains('Brazil'), 'Location'] = 'Brazil'

In [46]:
df1 = df[df['Location'] == 'Brazil']
df1.head()

,Clade,Location,Collection date
28128,O,Brazil,2021/10
36319,GK,Brazil,2021/08
37344,GK,Brazil,2021/08
37347,GK,Brazil,2021/08
42336,GR,Brazil,2020/10


In [47]:
#create cumulative sum of clade submission counts per month
df1 = df1.groupby(['Clade', 'Collection date','Location']).size().unstack(fill_value = 0)
# print(df1)
df1 = df1.reset_index()
df1.head()

Location,Clade,Collection date,Brazil
0,G,2020/01,1
1,G,2020/02,3
2,G,2020/03,105
3,G,2020/04,97
4,G,2020/05,39


In [48]:
df_global = df1.copy()
df_slide = df1.copy()

In [49]:
df_global = df_global.groupby('Clade').cumsum()
# print(df_global)
#add columns back
df_global['Collection date'] = df1['Collection date']
# print(df_global)
df_global['Clade'] = df1['Clade']
df_global[df_global['Clade'] == 'GRA']

Location,Brazil,Collection date,Clade
87,1,2021/01,GRA
88,2,2021/06,GRA
89,3,2021/07,GRA
90,4,2021/08,GRA
91,10,2021/11,GRA
92,2960,2021/12,GRA
93,22558,2022/01,GRA
94,33119,2022/02,GRA
95,37756,2022/03,GRA
96,39387,2022/04,GRA


In [50]:
# #function to calculate the log transformed ratio of two clades
# def transform_ratio(clade_1, clade_2):
#     return math.log10(clade_1/clade_2)

In [51]:
df_slide = df_slide[df_slide['Clade'] != 'GKA']
df_slide = df_slide[df_slide['Clade'] != 'GV']
df_slide = df_slide[df_slide['Clade'] != 'O']
df_slide = df_slide[df_slide['Clade'] != 'G']
df_slide = df_slide[df_slide['Clade'] != 'S']
df_slide = df_slide[df_slide['Clade'] != 'L']
df_slide = df_slide[df_slide['Clade'] != 'V']
df_slide.head()

Location,Clade,Collection date,Brazil
27,GH,2020/03,34
28,GH,2020/04,17
29,GH,2020/05,6
30,GH,2020/08,17
31,GH,2020/11,5


In [52]:
df_BR_qrt = df_slide[['Brazil', 'Clade', 'Collection date']]
df_BR_qrt.rename(columns = {'Brazil':'Values'}, inplace = True)
df_BR_qrt.head()
print(df_BR_qrt.shape)

(81, 3)


In [25]:
def quarterly_sum(dataframe):
    dataframe = dataframe.pivot(index = 'Collection date', columns = 'Clade', values = 'Values')
    dataframe = dataframe.replace([0, np.inf, np.nan], 1)
    dataframe = dataframe.reset_index()
    print(dataframe.head())
    
    dataframe.iloc[1,1:] = dataframe.iloc[0,1:] + dataframe.iloc[1,1:]
    print(dataframe.head())
    dataframe.iloc[2,1:] = dataframe.iloc[1,1:] + dataframe.iloc[2,1:]
    print(dataframe.head())
    dataframe.iloc[4,1:] = dataframe.iloc[3,1:] + dataframe.iloc[4,1:]
    print(dataframe.head())
    dataframe.iloc[5,1:] = dataframe.iloc[4,1:] + dataframe.iloc[5,1:]
    print(dataframe.head())
    
    dataframe.iloc[7,1:] = dataframe.iloc[6,1:] + dataframe.iloc[7,1:]
    dataframe.iloc[8,1:] = dataframe.iloc[7,1:] + dataframe.iloc[8,1:]
    
    dataframe.iloc[10,1:] = dataframe.iloc[9,1:] + dataframe.iloc[10,1:]
    dataframe.iloc[11,1:] = dataframe.iloc[10,1:] + dataframe.iloc[11,1:]
    
    dataframe.iloc[13,1:] = dataframe.iloc[12,1:] + dataframe.iloc[13,1:]
    dataframe.iloc[14,1:] = dataframe.iloc[13,1:] + dataframe.iloc[14,1:]
    
    dataframe.iloc[16,1:] = dataframe.iloc[15,1:] + dataframe.iloc[16,1:]
    dataframe.iloc[17,1:] = dataframe.iloc[16,1:] + dataframe.iloc[17,1:]
    
    dataframe.iloc[19,1:] = dataframe.iloc[18,1:] + dataframe.iloc[19,1:]
    dataframe.iloc[20,1:] = dataframe.iloc[19,1:] + dataframe.iloc[20,1:]
    
    dataframe.iloc[22,1:] = dataframe.iloc[21,1:] + dataframe.iloc[22,1:]
    dataframe.iloc[23,1:] = dataframe.iloc[22,1:] + dataframe.iloc[23,1:]
    print(dataframe)
    
    return dataframe

In [26]:
df_BR_qrt = quarterly_sum(df_BR_qrt)
df_BR_qrt

Clade Collection date    GH   GK      GR  GRA  GRY
0             2020/01   1.0  1.0     9.0  1.0  1.0
1             2020/02   1.0  1.0     3.0  1.0  1.0
2             2020/03  34.0  1.0   367.0  1.0  1.0
3             2020/04  17.0  1.0  1001.0  1.0  1.0
4             2020/05   6.0  1.0   434.0  1.0  1.0
Clade Collection date    GH   GK      GR  GRA  GRY
0             2020/01   1.0  1.0     9.0  1.0  1.0
1             2020/02   2.0  2.0    12.0  2.0  2.0
2             2020/03  34.0  1.0   367.0  1.0  1.0
3             2020/04  17.0  1.0  1001.0  1.0  1.0
4             2020/05   6.0  1.0   434.0  1.0  1.0
Clade Collection date    GH   GK      GR  GRA  GRY
0             2020/01   1.0  1.0     9.0  1.0  1.0
1             2020/02   2.0  2.0    12.0  2.0  2.0
2             2020/03  36.0  3.0   379.0  3.0  3.0
3             2020/04  17.0  1.0  1001.0  1.0  1.0
4             2020/05   6.0  1.0   434.0  1.0  1.0
Clade Collection date    GH   GK      GR  GRA  GRY
0             2020/01   1.0  1.

Clade,Collection date,GH,GK,GR,GRA,GRY
0,2020/01,1.0,1.0,9.0,1.0,1.0
1,2020/02,2.0,2.0,12.0,2.0,2.0
2,2020/03,36.0,3.0,379.0,3.0,3.0
3,2020/04,17.0,1.0,1001.0,1.0,1.0
4,2020/05,23.0,2.0,1435.0,2.0,2.0
5,2020/06,24.0,3.0,1951.0,3.0,3.0
6,2020/07,1.0,1.0,578.0,1.0,1.0
7,2020/08,18.0,2.0,1085.0,2.0,2.0
8,2020/09,19.0,3.0,1626.0,3.0,3.0
9,2020/10,1.0,1.0,572.0,1.0,1.0


In [27]:
#clades contains all GISAID clades, VOC contains GISAID clades containing VOCs
#change clade input to whichever you want to analyze
clades = ['G','GH','GK','GR','GRA','GRY','GV','L','O','S','V']
VOC = ['GH','GK','GR','GRA','GRY']

clade_input = VOC    ####

#function to generate distance matrix based on quarter, year, and continent
def create_distance_matrix(df,quarter_continent):
    tmp = df['Clade'].str.split('vs', expand=True)
    clade_list = tmp[0].unique()
    distance_matrix = pd.DataFrame(index = clade_list)
    distance_matrix[clade_list] = np.nan
    #fill in distance matrix
    for i in clade_list:
        for j in clade_list:
            if i == j:
                distance_matrix.loc[i,j] = 0
            else:
                distance_matrix.loc[i,j] = (abs((df.loc[df['Clade'] == '{a}vs{b}'.format(a=i,b=j),
                                                        quarter_continent].iloc[0] + 
                                            df.loc[df['Clade'] == '{a}vs{b}'.format(a=i,b=j), 
                                                   quarter_continent].iloc[0])/2))
                distance_matrix.loc[j,i] = (abs((df.loc[df['Clade'] == '{a}vs{b}'.format(a=i,b=j), 
                                                        quarter_continent].iloc[0] + 
                                            df.loc[df['Clade'] == '{a}vs{b}'.format(a=i,b=j), 
                                                   quarter_continent].iloc[0])/2))
    return distance_matrix

#function to generate difference matrix based on quarter, year, and continent
def create_difference_matrix(df,quarter_continent):
    tmp = df['Clade'].str.split('vs', expand=True)
    clade_list = tmp[0].unique()
    difference_matrix = pd.DataFrame(index = clade_list)
    difference_matrix[clade_list] = np.nan
    #fill in difference matrix
    for i in clade_list:
        for j in clade_list:
            if i == j:
                difference_matrix.loc[i,j] = 0
            else:
                difference_matrix.loc[i,j] = (abs((df.loc[df['Clade'] == '{a}vs{b}'.format(a=i,b=j),
                                                        quarter_continent].iloc[0] + 
                                            df.loc[df['Clade'] == '{a}vs{b}'.format(a=i,b=j), 
                                                   quarter_continent].iloc[0])/2))
                difference_matrix.loc[j,i] = (abs((df.loc[df['Clade'] == '{a}vs{b}'.format(a=i,b=j), 
                                                        quarter_continent].iloc[0] + 
                                            df.loc[df['Clade'] == '{a}vs{b}'.format(a=i,b=j), 
                                                   quarter_continent].iloc[0])/2))
    #if row_vs_col is a negative value, make it negative in difference matrix
    for i,v in difference_matrix.iterrows():
        for j in difference_matrix:
            if i != j:
                if (df.loc[df['Clade'] == '{a}vs{b}'.format(a=i,b=j),quarter_continent].iloc[0]) < 0:
                    difference_matrix.loc[i,j] *= -1
    return difference_matrix

#function to calculate the log transformed ratio of two clades
def transform_ratio(clade_1, clade_2):
    return np.log10(clade_1/clade_2)

def time_frame(df,time_window):
    dataframe = df.loc[df['Collection date'].isin(time_window)]
    dataframe.iloc[:,2:] = dataframe.iloc[:,2:].cumsum()
    for i in dataframe.columns.tolist():
        if dataframe[i].isnull().all():
            dataframe = dataframe.drop(i, axis=1)
    return dataframe

def time_frame_no_cumsum(df,time_window):
    dataframe = df.loc[df['Collection date'].isin(time_window)]
    return dataframe

#function returns slope of clade advantage plots
def slope_calc(day, clades):
    par = np.polyfit(day, clades, 1, full = True)
    slope= par[0][0]
    return slope

def clade_comparisons(df):
    clade_list = [i for i in clade_input if i in df.columns.tolist()]
    df['date_ordinal'] = pd.to_datetime(df['Collection date']).apply(lambda date: date.toordinal())
    df['days'] = df['date_ordinal'] - 737425
    #calculate log ratio for each clade comparison
    for i in clade_list:
        for j in clade_list:
            if i != j:
                df['{a}vs{b}'.format(a=i,b=j)] = df.apply(lambda row : 
                                                                        transform_ratio(row['{a}'.format(a=i)],
                                                                                        row['{b}'.format(b=j)]),
                                                                                        axis = 1)
    clade_list.append('date_ordinal')
    df = df.drop(clade_list, axis=1)
    return df

In [28]:
pd.set_option("display.max_rows", None, "display.max_columns", None)

In [29]:
df_BR_qrt = clade_comparisons(df_BR_qrt)
df_BR_qrt

Clade,Collection date,days,GHvsGK,GHvsGR,GHvsGRA,GHvsGRY,GKvsGH,GKvsGR,GKvsGRA,GKvsGRY,GRvsGH,GRvsGK,GRvsGRA,GRvsGRY,GRAvsGH,GRAvsGK,GRAvsGR,GRAvsGRY,GRYvsGH,GRYvsGK,GRYvsGR,GRYvsGRA
0,2020/01,0,0.000000,-0.954243,0.000000,0.000000,0.000000,-0.954243,0.000000,0.000000,0.954243,0.954243,0.954243,0.954243,0.000000,0.000000,-0.954243,0.000000,0.000000,0.000000,-0.954243,0.000000
1,2020/02,31,0.000000,-0.778151,0.000000,0.000000,0.000000,-0.778151,0.000000,0.000000,0.778151,0.778151,0.778151,0.778151,0.000000,0.000000,-0.778151,0.000000,0.000000,0.000000,-0.778151,0.000000
2,2020/03,60,1.079181,-1.022337,1.079181,1.079181,-1.079181,-2.101518,0.000000,0.000000,1.022337,2.101518,2.101518,2.101518,-1.079181,0.000000,-2.101518,0.000000,-1.079181,0.000000,-2.101518,0.000000
3,2020/04,91,1.230449,-1.769985,1.230449,1.230449,-1.230449,-3.000434,0.000000,0.000000,1.769985,3.000434,3.000434,3.000434,-1.230449,0.000000,-3.000434,0.000000,-1.230449,0.000000,-3.000434,0.000000
4,2020/05,121,1.060698,-1.795124,1.060698,1.060698,-1.060698,-2.855822,0.000000,0.000000,1.795124,2.855822,2.855822,2.855822,-1.060698,0.000000,-2.855822,0.000000,-1.060698,0.000000,-2.855822,0.000000
5,2020/06,152,0.903090,-1.910046,0.903090,0.903090,-0.903090,-2.813136,0.000000,0.000000,1.910046,2.813136,2.813136,2.813136,-0.903090,0.000000,-2.813136,0.000000,-0.903090,0.000000,-2.813136,0.000000
6,2020/07,182,0.000000,-2.761928,0.000000,0.000000,0.000000,-2.761928,0.000000,0.000000,2.761928,2.761928,2.761928,2.761928,0.000000,0.000000,-2.761928,0.000000,0.000000,0.000000,-2.761928,0.000000
7,2020/08,213,0.954243,-1.780157,0.954243,0.954243,-0.954243,-2.734400,0.000000,0.000000,1.780157,2.734400,2.734400,2.734400,-0.954243,0.000000,-2.734400,0.000000,-0.954243,0.000000,-2.734400,0.000000
8,2020/09,244,0.801632,-1.932367,0.801632,0.801632,-0.801632,-2.733999,0.000000,0.000000,1.932367,2.733999,2.733999,2.733999,-0.801632,0.000000,-2.733999,0.000000,-0.801632,0.000000,-2.733999,0.000000
9,2020/10,274,0.000000,-2.757396,0.000000,0.000000,0.000000,-2.757396,0.000000,0.000000,2.757396,2.757396,2.757396,2.757396,0.000000,0.000000,-2.757396,0.000000,0.000000,0.000000,-2.757396,0.000000


In [30]:
q1_2020_BR = []
q2_2020_BR = []
q3_2020_BR = []
q4_2020_BR = []
q1_2021_BR = []
q2_2021_BR = []
q3_2021_BR = []
q4_2021_BR = []

In [31]:
for i in range(20):
    q1_2020_BR.append(slope_calc(df_BR_qrt.iloc[0:3,1],df_BR_qrt.iloc[0:3,i + 2]))
    q2_2020_BR.append(slope_calc(df_BR_qrt.iloc[0:3,1],df_BR_qrt.iloc[3:6,i + 2]))
    q3_2020_BR.append(slope_calc(df_BR_qrt.iloc[0:3,1],df_BR_qrt.iloc[6:9,i + 2]))
    q4_2020_BR.append(slope_calc(df_BR_qrt.iloc[0:3,1],df_BR_qrt.iloc[9:12,i + 2]))
    q1_2021_BR.append(slope_calc(df_BR_qrt.iloc[0:3,1],df_BR_qrt.iloc[12:15,i + 2]))
    q2_2021_BR.append(slope_calc(df_BR_qrt.iloc[0:3,1],df_BR_qrt.iloc[15:18,i + 2]))
    q3_2021_BR.append(slope_calc(df_BR_qrt.iloc[0:3,1],df_BR_qrt.iloc[18:21,i + 2]))
    q4_2021_BR.append(slope_calc(df_BR_qrt.iloc[0:3,1],df_BR_qrt.iloc[21:24,i + 2]))

In [32]:
quarterly_slopes = pd.DataFrame()

quarterly_slopes['q1_2020_BR'] = q1_2020_BR
quarterly_slopes['q2_2020_BR'] = q2_2020_BR
quarterly_slopes['q3_2020_BR'] = q3_2020_BR
quarterly_slopes['q4_2020_BR'] = q4_2020_BR
quarterly_slopes['q1_2021_BR'] = q1_2021_BR
quarterly_slopes['q2_2021_BR'] = q2_2021_BR
quarterly_slopes['q3_2021_BR'] = q3_2021_BR
quarterly_slopes['q4_2021_BR'] = q4_2021_BR

quarterly_slopes.index = df_BR_qrt.columns[2:]

In [33]:
quarterly_slopes['Clade'] = quarterly_slopes.index

In [34]:
quarterly_slopes.head()

,q1_2020_BR,q2_2020_BR,q3_2020_BR,q4_2020_BR,q1_2021_BR,q2_2021_BR,q3_2021_BR,q4_2021_BR,Clade
Clade,,,,,,,,,
GHvsGK,0.017780,-0.005456,0.013560,0.008791,0.004978,-0.015417,-0.009137,0.009542,GHvsGK
GHvsGR,-0.001057,-0.002317,0.014031,0.005252,-0.003400,-0.000249,0.006113,0.012468,GHvsGR
GHvsGRA,0.017780,-0.005456,0.013560,0.008791,-0.000057,0.000689,0.001577,-0.041441,GHvsGRA
GHvsGRY,0.017780,-0.005456,0.013560,0.001773,-0.003643,0.000814,0.006945,0.008084,GHvsGRY
GKvsGH,-0.017780,0.005456,-0.013560,-0.008791,-0.004978,0.015417,0.009137,-0.009542,GKvsGH


In [35]:
quarterly_slopes = quarterly_slopes.melt(id_vars=['Clade'],
       var_name='Location',
       value_name='Slope')

In [36]:
quarterly_slopes[['ID1','ID2']] = quarterly_slopes.Clade.str.split('vs',expand=True)
quarterly_slopes.head()

,Clade,Location,Slope,ID1,ID2
0,GHvsGK,q1_2020_BR,0.017780,GH,GK
1,GHvsGR,q1_2020_BR,-0.001057,GH,GR
2,GHvsGRA,q1_2020_BR,0.017780,GH,GRA
3,GHvsGRY,q1_2020_BR,0.017780,GH,GRY
4,GKvsGH,q1_2020_BR,-0.017780,GK,GH


In [37]:
quarterly_slopes = quarterly_slopes.drop(['Clade'], axis=1)
quarterly_slopes.head()

,Location,Slope,ID1,ID2
0,q1_2020_BR,0.017780,GH,GK
1,q1_2020_BR,-0.001057,GH,GR
2,q1_2020_BR,0.017780,GH,GRA
3,q1_2020_BR,0.017780,GH,GRY
4,q1_2020_BR,-0.017780,GK,GH


In [38]:
quarterly_slopes['Quarter_Year'] = quarterly_slopes.Location.str.split('_',2)
quarterly_slopes['Location'] = quarterly_slopes.Quarter_Year.apply(lambda x: x[-1])
quarterly_slopes['Quarter_Year'] = quarterly_slopes.Quarter_Year.apply(lambda x: x[:2])
quarterly_slopes.head()

,Location,Slope,ID1,ID2,Quarter_Year
0,BR,0.017780,GH,GK,"[q1, 2020]"
1,BR,-0.001057,GH,GR,"[q1, 2020]"
2,BR,0.017780,GH,GRA,"[q1, 2020]"
3,BR,0.017780,GH,GRY,"[q1, 2020]"
4,BR,-0.017780,GK,GH,"[q1, 2020]"


In [39]:
quarterly_slopes['Quarter_Year'] = quarterly_slopes['Quarter_Year'].str.join('_')
quarterly_slopes.head()

,Location,Slope,ID1,ID2,Quarter_Year
0,BR,0.017780,GH,GK,q1_2020
1,BR,-0.001057,GH,GR,q1_2020
2,BR,0.017780,GH,GRA,q1_2020
3,BR,0.017780,GH,GRY,q1_2020
4,BR,-0.017780,GK,GH,q1_2020


In [40]:
quarterly_slopes = quarterly_slopes[['ID1','ID2','Slope','Location','Quarter_Year']]
quarterly_slopes.head()

,ID1,ID2,Slope,Location,Quarter_Year
0,GH,GK,0.017780,BR,q1_2020
1,GH,GR,-0.001057,BR,q1_2020
2,GH,GRA,0.017780,BR,q1_2020
3,GH,GRY,0.017780,BR,q1_2020
4,GK,GH,-0.017780,BR,q1_2020


In [41]:
quarterly_slopes[['Quarter','Year']] = quarterly_slopes['Quarter_Year'].str.split('_', expand=True)
quarterly_slopes.head()

,ID1,ID2,Slope,Location,Quarter_Year,Quarter,Year
0,GH,GK,0.017780,BR,q1_2020,q1,2020
1,GH,GR,-0.001057,BR,q1_2020,q1,2020
2,GH,GRA,0.017780,BR,q1_2020,q1,2020
3,GH,GRY,0.017780,BR,q1_2020,q1,2020
4,GK,GH,-0.017780,BR,q1_2020,q1,2020


In [42]:
quarterly_slopes = quarterly_slopes.drop(['Quarter_Year'], axis=1)
quarterly_slopes.head()

,ID1,ID2,Slope,Location,Quarter,Year
0,GH,GK,0.017780,BR,q1,2020
1,GH,GR,-0.001057,BR,q1,2020
2,GH,GRA,0.017780,BR,q1,2020
3,GH,GRY,0.017780,BR,q1,2020
4,GK,GH,-0.017780,BR,q1,2020
